In [ ]:
import c3d
from c3d import *

feature_extractor = c3d_feature_extractor()

In [ ]:
import os
import cv2, pickle
import numpy as np
import random, asyncio
from tensorflow.keras.models import load_model

In [ ]:
clf = load_model("video.h5", compile=False)
le = pickle.load(open("labels.pkl",'rb'))

In [ ]:
async def extract_frame(cap, frame_number):
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()
    if ret:
        frame = cv2.resize(frame, (480, 640))
        return frame

In [ ]:
async def extract_frames_async(video_path, num_frames):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_numbers = [int(i * total_frames / num_frames) for i in range(num_frames)]
    frame_numbers.sort()
    tasks = []
    for frame_number in frame_numbers:
        task = extract_frame(cap, frame_number)
        tasks.append(task)
    frames = await asyncio.gather(*tasks)
    input = np.array(frames)
    clip = preprocess_input(input)
    rgb_feature = feature_extractor.predict(clip)[0]
    new_feature = np.expand_dims(rgb_feature,axis=0)
    pred=clf.predict(new_feature)[0]
    cap.release()
    return {le.inverse_transform([np.argmax(pred)])[0]: pred[np.argmax(pred)]}

In [ ]:
video_path = "path_to_video/video.mp4"
num_frames = 256

loop = asyncio.get_event_loop()
label,score = loop.run_until_complete(extract_frames_async(video_path, num_frames))
loop.close()